# ROUGH DRAFT 1

# Capstone Project - Battle of the Neighborhoods
### Applied Data Science Capstone - IBM Coursera

### Table of Contents

* [1. Introduction: Business Problem](#introduction)
* [2. Data Description](#data)
* [3. Methodology](#methodology)
* [4. Analysis](#analysis)
* [5. Results and Discussion](#results)
* [6. Conclusion](#conclusion)

<a id="introduction"></a>

### 1. Introduction: Business Problem - Opening an Argentinean Restaurant or Cafe

This project will focus around two major cities. Cities known for diversity. With diversity, brings diverse resturaunts, cafes and more, which bring people together. If you're a stakeholder, entrepreneur, or chef interested in opening an Argentinean resturaunt or cafe in New York city or Toronto, this will be for you. 

Restaurant industry is huge in both of these cities, so we will try to identify the best location. To find the best location, we will use a number of data points such as: popular visiting locations, population percentage per borough, amount of current vertical competition, and no argentinean restaurants in the area. 

Using efficient data science tools, we will generate two promisions locations for both major cities. Each location will have the benefits explained and can be decided by the stakeholder.


<a id="data"></a>

### 2. Data Description

The expected data to be applied for our decision are: 
* Top five venues visited per neighborgood(borough)
* Population percentage per borough
* Amount of competition (Restaurant & Cafes) per neighborhood
* Current location and foursquare information for Argentinean Resturaunts

We will attempt to seperate neighborhoods using Choropleth maps and/or by radius. 

The following data sources will be utilized to extract and generate the required information. The list below represents the order in which the data will be generated. 

* New York City Census Information. CSV.
* Canada Census 2016: Population & Density information
* Wiki: Toronto Postal Codes
* Foursquare API: Retreive results for amount of restaurants and cafes, check in information,  popularity to provide competition insight.
* Google Maps API Geocode: Product location information, visualize locations for comparision, provide results.

In [14]:
########################################
#### Libraries used in this section ####
########################################

import numpy as np
import pandas as pd # library for data analsysis
import requests
from bs4 import BeautifulSoup
import config
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # plotting library
from sklearn.cluster import KMeans # import k-means from clustering stage
import matplotlib
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors # Matplotlib and associated plotting modules

#### 2a. New York City Census Information

New York City Census Information will be used to gather the population data for each county located in New York City. County population data will help determine which area we select for a future resturaunt location. 

In [15]:
# Import: NYC Census Data CSV
links={'pop':'C:/Users/pompa/gitProjects/Coursera_Capstone/Coursera_Capstone/QuickFactsFeb2020.csv'}
# Create: DataFrame with Census Data
df_POP = pd.read_csv(links["pop"]) 

In [16]:
# Verify Data: Identify what needs to be removed.
df_POP.head()

,Fact,Fact Note,"New York city, New York","Value Note for New York city, New York","Bronx County (Bronx Borough), New York","Value Note for Bronx County (Bronx Borough), New York","Kings County (Brooklyn Borough), New York","Value Note for Kings County (Brooklyn Borough), New York","New York County (Manhattan Borough), New York","Value Note for New York County (Manhattan Borough), New York","Queens County (Queens Borough), New York","Value Note for Queens County (Queens Borough), New York","Richmond County (Staten Island Borough), New York","Value Note for Richmond County (Staten Island Borough), New York"
0,"Population estimates, July 1, 2019, (V2019)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Population estimates, July 1, 2018, (V2018)",NaN,"8,398,748",NaN,"1,432,132",NaN,"2,582,830",NaN,"1,628,701",NaN,"2,278,906",NaN,"476,179",NaN
2,"Population estimates base, April 1, 2010, (V2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Population estimates base, April 1, 2010, (V2...",NaN,"8,174,988",NaN,"1,384,603",NaN,"2,504,717",NaN,"1,586,360",NaN,"2,230,578",NaN,"468,730",NaN
4,"Population, percent change - April 1, 2010 (es...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Clean Data: Drop NaN columns not needed.
df_POP.drop("Fact Note", axis = 1, inplace=True)
df_POP.drop("Value Note for New York city, New York", axis = 1, inplace=True)
df_POP.drop("Value Note for Bronx County (Bronx Borough), New York", axis = 1, inplace=True)
df_POP.drop("Value Note for Kings County (Brooklyn Borough), New York", axis = 1, inplace=True)
df_POP.drop("Value Note for New York County (Manhattan Borough), New York", axis = 1, inplace=True)
df_POP.drop("Value Note for Queens County (Queens Borough), New York", axis = 1, inplace=True)
df_POP.drop("Value Note for Richmond County (Staten Island Borough), New York", axis = 1, inplace=True)
df_POP.drop([2, 4], axis = 0, inplace=True)

In [18]:
# Clean Data: Drop extra row
df_POP.drop(0, axis = 0, inplace=True)
df_POP.head()

,Fact,"New York city, New York","Bronx County (Bronx Borough), New York","Kings County (Brooklyn Borough), New York","New York County (Manhattan Borough), New York","Queens County (Queens Borough), New York","Richmond County (Staten Island Borough), New York"
1,"Population estimates, July 1, 2018, (V2018)","8,398,748","1,432,132","2,582,830","1,628,701","2,278,906","476,179"
3,"Population estimates base, April 1, 2010, (V2...","8,174,988","1,384,603","2,504,717","1,586,360","2,230,578","468,730"
5,"Population, percent change - April 1, 2010 (es...",2.7%,3.4%,3.1%,2.7%,2.2%,1.6%
6,"Population, Census, April 1, 2010","8,175,133","1,385,108","2,504,700","1,585,873","2,230,722","468,730"
7,"Persons under 5 years, percent",6.5%,7.2%,7.2%,4.7%,6.2%,5.8%


In [19]:
# Clean Data: Rename Columns, drop data.
df_POP.columns = ['2018', 'New York city', 'Bronx County', 'Kings County', 'New York County', 'Queens County', 'Richmond County']
df_POP2 = df_POP
df_POP2 = df_POP2[:1]
df_POP2.drop("2018", axis = 1, inplace=True)

C:\Users\pompa\Anaconda3\envs\ipykernel_py2\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


#### 2b. New York City population & population per county.

In [20]:
# Second df with only population
df_POP2

,New York city,Bronx County,Kings County,New York County,Queens County,Richmond County
1,"8,398,748","1,432,132","2,582,830","1,628,701","2,278,906","476,179"


In [21]:
# Filtered: pop by county in NYC.
nyc_Totalpop = df_POP2['New York city'].str.replace(',', '').astype(int)
bronxCounty = df_POP2['Bronx County'].str.replace(',', '').astype(int)
kingsCounty = df_POP2['Kings County'].str.replace(',', '').astype(int)
nyCounty = df_POP2['New York County'].str.replace(',', '').astype(int)
queensCounty = df_POP2['Queens County'].str.replace(',', '').astype(int)
richmondCounty = df_POP2['Richmond County'].str.replace(',', '').astype(int)

In [22]:
# Filtered: Percentage of population per County
bronxCounty_popPercentage = int(bronxCounty) / int(nyc_Totalpop)
kingsCounty_popPercentage = int(kingsCounty) /  int(nyc_Totalpop)
nyCounty_popPercentage = int(nyCounty) / int(nyc_Totalpop)
queensCounty_popPercentage = int(queensCounty) / int(nyc_Totalpop)
richmondCounty_popPercentage = int(richmondCounty) / int(nyc_Totalpop)

In [23]:
# Clean: Initialise data to lists. 
data = [{'Bronx County': bronxCounty_popPercentage, 'Kings County': kingsCounty_popPercentage, 'New York County': nyCounty_popPercentage, 'Queens County': queensCounty_popPercentage, 'Richmond County': richmondCounty_popPercentage}] 

# Creates Percentage of population per County DataFrame. 
df_popPercentage = pd.DataFrame(data)
df_popPercentage

,Bronx County,Kings County,New York County,Queens County,Richmond County
0,0.170517,0.307526,0.193922,0.271339,0.056696


#### 2c. Latitude/Longitude for NYC counties.

In [24]:
### Lat Lng of Boroughs in New York City
# Filter: Used 'address'
address = 'Queens, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7498243 -73.7976337


In [25]:
# Create DataFrame with Lat/Lng for each borough in NYC.
latlng_nyc ={'Bronx County':['40.850485', '-73.840403'], 'Kings County':['40.645309', '-73.955023'], 'New York County':['40.78962', '-73.95989'], 'Queens County':['40.7498', '-73.7976'], 'Richmond County':['40.564209', '-74.125304']} 
df_NYCB_latlng = pd.DataFrame(latlng_nyc, index =['latitude', 'longitude']) 
df_NYCB_latlng = df_NYCB_latlng.astype('float')
df_NYCB_latlng

,Bronx County,Kings County,New York County,Queens County,Richmond County
latitude,40.850485,40.645309,40.78962,40.7498,40.564209
longitude,-73.840403,-73.955023,-73.95989,-73.7976,-74.125304


In [ ]:
###################################################################################

#### 2d. Search for a Neighborhood in Toronto

In [91]:
# Import Wiki table information.
weblink = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(weblink.content,'lxml')
table = soup.find_all('tbody')[0]

# Select Table with data
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')

### Data for table
data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    data.append(row)
#print(data)
#print(df[0].to_json(orient='records'))

### Convert imported data to pandas DataFrame
df = pd.DataFrame(data, columns=["Postcode", "Borough", "Neighbourhood"])

#### Clean Data

In [92]:
# Remove \n
df = df.replace(r'\n','', regex=True)
df.columns = df.columns.str.strip()
# verify column headers
#df.columns

# verify column headers
df.columns

# Group columns by postal code and borough, removing duplicate postcode value
# and moving the duplicate data in borough to Neighbourhood.
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

# Rename headers
df.rename(columns ={'Postcode': 'PostalCode'}, inplace=True)
df.rename(columns ={'Neighbourhood': 'Neighborhood'}, inplace=True)

# Verify neighbourhood has values in all rows.
df.isin(['Not Available']).any().any()

False

#### Generating Lat & Lng by Postal Code.

In [22]:
# Main list to run
postal_code = df['PostalCode'].tolist()

In [23]:
# Main list
list1 = postal_code
# Getting length of list 
api_key = config.api_key
i = 0
# Iterating using while loop 
while i < len(list1):  
    element = list1[i]
    geoCodeUrl = "https://maps.googleapis.com/maps/api/geocode/json?components=postal_code:{}|country:CA&key={}".format(
        element,
        api_key) 
    
    # make the GET request
    lookup = requests.get(geoCodeUrl)
    data = lookup.json()  
    
    # If no results are found for postal code, skip and move on.
    if (data['status'] == 'ZERO_RESULTS'):
        i += 1
        continue
    # Instantiate an empty dict
    latlng = {}
    # latlng information
    latlng['PostalCode'] = data['results'][0]['address_components'][0]['long_name']
    latlng['Longname'] = data['results'][0]['address_components'][1]['long_name']
    latlng['Latitude'] = data['results'][0]['geometry']['location']['lat']
    latlng['Longitude'] = data['results'][0]['geometry']['location']['lng']

    with open('data.json', 'r') as j:
        json_data = json.load(j)
    # convert data to list if not
        if type(json_data) is dict:
            json_data = [json_data]

    # use append() to add to list
    json_data.append(latlng)    

    #write list to file
    with open('data.json', 'w') as outfile:
        json.dump(json_data, outfile)
    # Normalize data. Flatten JSON.
    data_normalized = pd.json_normalize(json_data) # flatten JSON
    i += 1 
       
    if i == len(list1):
        break

print(data_normalized)

    PostalCode     Longname   Latitude  Longitude venues
0          M1J  Scarborough  43.744734 -79.239476    NaN
1          M1B  Scarborough  43.806686 -79.194353    NaN
2          M1C  Scarborough  43.784535 -79.160497    NaN
3          M1E  Scarborough  43.763573 -79.188711    NaN
4          M1G  Scarborough  43.770992 -79.216917    NaN
..         ...          ...        ...        ...    ...
234        M9N      Toronto  43.706876 -79.518188    NaN
235        M9P    Etobicoke  43.696319 -79.532242    NaN
236        M9R    Etobicoke  43.688905 -79.554724    NaN
237        M9V    Etobicoke  43.739416 -79.588437    NaN
238        M9W    Etobicoke  43.706748 -79.594054    NaN

[239 rows x 5 columns]


In [93]:
# Open new json file created above.
with open('data.json', 'r') as j:
    json_data = json.load(j)

In [94]:
# Rename longname to borough on new generated dataframe
data_df = pd.DataFrame(json_data)
data_df.rename(columns = {'Longname':'Borough'}, inplace=True)
data_df

,PostalCode,Borough,Latitude,Longitude,venues
0,M1J,Scarborough,43.744734,-79.239476,NaN
1,M1B,Scarborough,43.806686,-79.194353,NaN
2,M1C,Scarborough,43.784535,-79.160497,NaN
3,M1E,Scarborough,43.763573,-79.188711,NaN
4,M1G,Scarborough,43.770992,-79.216917,NaN


In [101]:
# Clean data: Remove dups, NaN
data_df = data_df.drop_duplicates(subset='Latitude', keep='first')
data_df = data_df.drop(['venues'], axis=1)
data_df = data_df.dropna(how='all')

In [102]:
data_df

,PostalCode,Borough,Latitude,Longitude
0,M1J,Scarborough,43.744734,-79.239476
1,M1B,Scarborough,43.806686,-79.194353
2,M1C,Scarborough,43.784535,-79.160497
3,M1E,Scarborough,43.763573,-79.188711
4,M1G,Scarborough,43.770992,-79.216917
...,...,...,...,...
96,M9N,Toronto,43.706876,-79.518188
97,M9P,Etobicoke,43.696319,-79.532242
98,M9R,Etobicoke,43.688905,-79.554724
99,M9V,Etobicoke,43.739416,-79.588437


In [17]:
####################################################################
#### Combine Dataframes: Postal Code, Neighborhood, with Latlng ####
####################################################################

In [103]:
# Merge df and df_data
# Postal code with 
df_merge_col = pd.merge(df, data_df, on='PostalCode')

# Clean df_merge_col
df_merge_col.rename(columns = {'Borough_x':'Borough'}, inplace=True)
df_merge_col.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace=True)
df_merge_col = df_merge_col.drop(['Borough_y'], axis=1)
df_merge_col.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<a id="methodology"></a>

### 3. Methodology

## FIX THIS. Not complete. needs to be in order.

The goal is to provide two possible locations for both New York City and Toronto, to build a restaurant and/or cafe. By focusing on high population density areas, popular areas for food, and no current Argentinean restaurants around.

First, we collect data from our sources such as: Neighborhood latitude and longitude information, number of resturaunts and cafes in both major cities, location and number of Argentinean resturaunts in both cities, and competition information. 

Second, we calculate the population density per neighborhood and create a heatmap showing density per neighborhood. We will also use heatmaps to distingush the amount of resturaunts and cafes per neighborgood, per city. 

Third, we will create clusters of neighborhoods which meet our requirements in providing promising locations to discuss with stakeholders. Using K-Means clustering, we will present a a map with the following locations found. 

Show: growth of population in counties per year. Possibly growth of selections

<a id="analysis"></a>

### 4. Analysis

Analysis will combine the data acquired in section #2, combine Google location data with Four Square information to find the solutions needed. Solutions include: Argentinian restauraunts per county in NYC and Toronto, competition in each area, and population percentage per county.

Using the location data acquired in section #2, lets use the Latitude, Longitude, and County information to <b>locate the number of Argentinian and Empanada restauraunts by NYC County.</b>

In [26]:
# Fourquare Information. 
VERSION = '20180604'
LIMIT = 30
radius = 6000
# Counties and Queeries being searched. 
## a / x = Counties ; b / y = queries
counties = ['Bronx', 'Kings', 'Brooklyn', 'Manhattan', 'Queens', 'Richmond']
queries = ['chimichurri', 'empanadas', 'Argentinian']
CLIENT_ID = config.CLIENT_ID
CLIENT_SECRET = config.CLIENT_SECRET

#### 4a. Search all counties in NYC. Search each queries in all counties. Generate an empty dataframe to import data into. Createnested df with all queries. In the end, we combine and clean up data. 

In [27]:
# Create an empty dataFrame to fill in next step. All counties will have their own DF and will
# be nested into one main. 
dataframe = {}
for city in counties:
    dataframe[city] = pd.DataFrame()

In [30]:
# Data: Loop through a number of search queries. We call the Four Square API to search 
# and query specific keywords related to Argentinian restauraunts.
# We will use this loop for each county to find data by specific keywords. 
# 'dataframe' is populated on every loop and will provide results of query per location requested.
i=0
for x, y in [(x,y) for x in counties for y in queries]:      
    for i in range(len(x)):
        element2 = x
        element = y
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, element2, VERSION, element, radius, LIMIT)
        results = requests.get(url).json()

        result = {}                                #list that will store the results of all Json 
        result = results['response']['venues']     # filter specific location in results.json   

        
        with open('databackup.json', 'r') as j:
            json_data = json.load(j)
        # convert data to list if not
            if type(json_data) is dict:
                json_data = [json_data]
        # use append() to add to list
        json_data.append(result)    
        #write list to file
        with open('databackup.json', 'w') as outfile:
            json.dump(json_data, outfile)    



        #dataframe[element2] = pd.DataFrame() 
        dataframe[element2] = dataframe[element2].append(result, ignore_index=True) # add filtered JSON data to empty dataframe.

        i += 1    # add 1 to i and restart loop
            
print('completed')

completed


In [32]:
# Single out nested dataFrames to concat after.
df_bronx = pd.DataFrame(dataframe['Bronx'])
df_kings = pd.DataFrame(dataframe['Kings'])
df_brooklyn = pd.DataFrame(dataframe['Brooklyn'])
df_nycounty = pd.DataFrame(dataframe['Manhattan'])
df_queens = pd.DataFrame(dataframe['Queens']) 

#df_richmond = pd.DataFrame(dataframe['Richmond']) no searches found

In [34]:
counties2 = [df_bronx, df_kings, df_brooklyn, df_nycounty, df_queens]
# Add City to each nested DataFrame in dataframe
i = 0
for i in range(len(counties2)):
    element2 = counties2[i]

    def get_city_type(row):
        try:
            city_list = row['location']      
        except:
            city_list = row['venue.location']        
        if len(city_list) == 0:
            return None
        else:
            return city_list['city']    
    element2['City'] = element2.apply(get_city_type, axis=1)
    i += 1
print('Completed')

Completed


In [55]:
# Remove unwanted neighborhoods from county dataframes
i = 0
for i in range(len(counties2)):
    element2 = counties2[i]
    element2 = element2.drop_duplicates(subset='id', keep='first')
    
    bronxNames = df_bronx[ df_bronx['City'] == 'New York' ].index
    kingsNames = df_kings[ (df_kings['City'] == 'New York') | (df_kings['City'] == 'Ridgewood') ].index
    brooklynNames = df_brooklyn[ (df_brooklyn['City'] == 'New York') | (df_brooklyn['City'] == 'Ridgewood') ].index
    nycountyNames = df_nycounty[ (df_nycounty['City'] == 'Union City') | (df_nycounty['City'] == 'Astoria') | (df_nycounty['City'] == 'Bronx') | (df_nycounty['City'] == 'North Bergen') | (df_nycounty['City'] == 'Brooklyn') | (df_nycounty['City'] == 'West New York') | (df_nycounty['City'] == 'Queens') ].index
    queensNames = df_queens[ (df_queens['City'] == 'Brooklyn') | (df_queens['City'] == 'New York') ].index

    if i == 0:
        #delete extra neighborhoods not in bronx counties
        df_bronx.drop(bronxNames , inplace=True)
        df_bronx['Cluster Labels']='0'
    if i == 1:
        #delete extra neighborhoods not in kings/brooklyn counties
        df_kings.drop(kingsNames , inplace=True)
        df_kings['Cluster Labels']='1'
    if i == 2:
        #delete extra neighborhoods not in brooklyn/kings counties
        df_brooklyn.drop(brooklynNames , inplace=True)
        df_brooklyn['Cluster Labels']='1'
    if i == 3:
        #delete extra neighborhoods not in NYC/Manhattan counties
        df_nycounty.drop(nycountyNames , inplace=True)
        df_nycounty['Cluster Labels']='2'
    if i == 4:
        #delete extra neighborhoods not in Queens counties
        df_queens.drop(queensNames , inplace=True)
        df_queens['Cluster Labels']='3'
        
    i += 1

print('Completed')    

Completed


In [56]:
## Group df_kings & df_brooklyn (same county/neighborhood)
## Delete duplicates

kings_brooklyn = [df_kings, df_brooklyn]
df_kingsBrooklyn = pd.concat(kings_brooklyn, ignore_index=True)

# Remove duplicates from concat df's
df_kingsBrooklyn = df_kingsBrooklyn.drop_duplicates(subset='id', keep='first')

In [57]:
# Group updated dataframes

Group_frames = [df_bronx, df_kingsBrooklyn, df_nycounty, df_queens]
result = pd.concat(Group_frames, ignore_index=True)

In [58]:
# Clean: data is arranged to provide name, categorie titles, latitude and longitude.
# 'dataframe' is filtered with def to extract information from values.

filtered_columns = ['name', 'categories', 'location', 'City', 'Cluster Labels'] + ['id']
dataframe_filtered = result.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']      
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def get_lat_type(row):
    try:
        lat_list = row['location']      
    except:
        lat_list = row['venue.location']        
    if len(lat_list) == 0:
        return None
    else:
        return lat_list['lat']

def get_lng_type(row):
    try:
        lng_list = row['location']      
    except:
        lng_list = row['venue.location']        
    if len(lng_list) == 0:
        return None
    else:
        return lng_list['lng']

In [59]:
# filter the category for each row

dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\.', '').str.replace(r'\$', '').str.replace(r'\'', '').str.replace(r'\&', 'and').str.replace(r'\?', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\$', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\'', '')


dataframe_filtered['name'].str.replace('.', '').astype(object)
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered['Latitude'] = dataframe_filtered.apply(get_lat_type, axis=1)
dataframe_filtered['Longitude'] = dataframe_filtered.apply(get_lng_type, axis=1)

dataframe_filtered = dataframe_filtered[['name', 'categories', 'location', 'Latitude', 'Longitude', 'City', 'Cluster Labels', 'id']]
dataframe_filtered = dataframe_filtered.drop_duplicates(subset='id', keep='first')

In [60]:
dataframe_filtered

,name,categories,location,Latitude,Longitude,City,Cluster Labels,id
0,Chimichurri Truck,Food Truck,"{'address': '204 street', 'crossStreet': '9th ...",40.836275,-73.887374,Bronx,0,4bfdc1ac4cf820a11bfeecf4
1,Chimichurri Truck,Food Truck,"{'address': 'West 174th street', 'crossStreet'...",40.826322,-73.923097,Bronx,0,4d8ffbfdc479a35d531a6997
2,La Chamaca Chimichurri,Food Truck,"{'lat': 40.846849, 'lng': -73.902605, 'labeled...",40.846849,-73.902605,Bronx,0,4eed70140e61a418653b7134
3,EL DISCIPULO Chimichurri Stand,Food Truck,"{'address': 'FORDHAM ROAD.', 'lat': 40.8619295...",40.861930,-73.910197,Bronx,0,4dd76ed3d1647fcf3e4ab344
20,Empanadas Monumental,Empanada Restaurant,"{'address': '366 E 204th St', 'crossStreet': '...",40.871607,-73.877352,Bronx,0,5209658e11d2e53485456870
21,Empanadas Monumental,Empanada Restaurant,"{'address': '1971 University Ave', 'lat': 40.8...",40.854293,-73.911540,Bronx,0,556ccf48498e6c4a3baa6c34
22,Empanadas Monumental,Empanada Restaurant,"{'address': '206 W 236th St', 'crossStreet': '...",40.883476,-73.901964,Bronx,0,5d1be938ab76360023fcd0d8
23,El Eden Empanadas,Paella Restaurant,"{'lat': 40.864413, 'lng': -73.891372, 'labeled...",40.864413,-73.891372,Bronx,0,533c777f498e3638af45325c
24,Empanadas Monumental,Empanada Restaurant,"{'address': '844 River Ave,', 'lat': 40.827348...",40.827348,-73.926283,Bronx,0,5423ab56498e7ef0755d8ff9
25,Tito Empanadas Y Mas,Spanish Restaurant,"{'address': '1221 Jerome Ave', 'lat': 40.83626...",40.836264,-73.922621,Bronx,0,525c31f0498ecec392c033ff


In [61]:
dataframe_filtered['categories'].value_counts()

Empanada Restaurant          23
Food Truck                   12
Latin American Restaurant     3
Spanish Restaurant            3
Steakhouse                    2
Mexican Restaurant            2
Food Stand                    1
Embassy / Consulate           1
Street Food Gathering         1
Breakfast Spot                1
Food                          1
Pizza Place                   1
Paella Restaurant             1
Cuban Restaurant              1
Name: categories, dtype: int64

In [62]:
#Removing unwanted category titles: 'Cuban Restaurant', 'Caribbean Restaurant', 'Mexican Restaurant' , 'Embassy / Consulate', 'Street Food Gathering'

indexNames = dataframe_filtered[ (dataframe_filtered['categories'] == 'Embassy / Consulate') | (dataframe_filtered['categories'] == 'Mexican Restaurant') | (dataframe_filtered['categories'] == 'Cuban Restaurant') | (dataframe_filtered['categories'] == 'Caribbean Restaurant') | (dataframe_filtered['categories'] == 'Street Food Gathering') ].index
dataframe_filtered.drop(indexNames, inplace=True)

In [63]:
dataframe_filtered['categories'].value_counts()

Empanada Restaurant          23
Food Truck                   12
Latin American Restaurant     3
Spanish Restaurant            3
Steakhouse                    2
Food Stand                    1
Paella Restaurant             1
Breakfast Spot                1
Food                          1
Pizza Place                   1
Name: categories, dtype: int64

#### 4b. Visualize: All counties combined.

In [64]:
# NYC County Lat/lng
nyclat = 40.7831 
nyclng = -73.9712

In [65]:
venues_map = folium.Map(location=[nyclat, nyclng], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the New York City
folium.features.CircleMarker(
    [nyclat, nyclng],
    radius=10,
    color='red',
    popup='New York City, NY',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add all restaurant results as blue circle markers
for lat, lng, label in zip(dataframe_filtered.Latitude, dataframe_filtered.Longitude, dataframe_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=3,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)

# display map
venues_map

#### 4c. Cluster neighborhoods in NYC. Generate map and visualize clusters. 

In [66]:
# set number of clusters
kclusters = 4
# Lat/Lngfor 
latitude = dataframe_filtered['Latitude'].tolist()
longitude = dataframe_filtered['Longitude'].tolist()

In [90]:
# create map
map_clusters = folium.Map(location=[nyclat, nyclng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.seismic(np.linspace(0, 1, len(ys)))
seismic = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dataframe_filtered['Latitude'], dataframe_filtered['Longitude'], dataframe_filtered['name'], dataframe_filtered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=seismic[int(cluster)-1],
        fill=True,
        fill_color=seismic[int(cluster)-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

#### 4d. View cluster information

In [ ]:
#Bronx cluster / Brown
dataframe_filtered.loc[dataframe_filtered['Cluster Labels'] == 0, dataframe_filtered.columns[[1] + list(range(5, dataframe_filtered.shape[1]))]]

In [ ]:
# Kings/Brooklyn County cluster / Black
dataframe_filtered.loc[dataframe_filtered['Cluster Labels'] == 1, dataframe_filtered.columns[[1] + list(range(5, dataframe_filtered.shape[1]))]]

In [ ]:
# NYC County cluster / light blue
dataframe_filtered.loc[dataframe_filtered['Cluster Labels'] == 2, dataframe_filtered.columns[[1] + list(range(5, dataframe_filtered.shape[1]))]]

In [ ]:
# Queens cluster / Pink-light Red
dataframe_filtered.loc[dataframe_filtered['Cluster Labels'] == 3, dataframe_filtered.columns[[1] + list(range(5, dataframe_filtered.shape[1]))]]

In [84]:
# Amount of Argentinian style restuarants in New York City, New York.
dataframe_filtered['name'].count()

51

In [ ]:
####################

#### 4e. Toronto restuarant search requests. 

In [ ]:
####################

ALL TORONTO DATA WILL BE LOADED HERE. Currently testing.

<a id="results"></a>

### Results & Discussion

In [ ]:
# Amount of Argentinian style restuarants in New York City, New York.
dataframe_filtered['name'].count()

In [89]:
df_POP2

,New York city,Bronx County,Kings County,New York County,Queens County,Richmond County
1,"8,398,748","1,432,132","2,582,830","1,628,701","2,278,906","476,179"


In [85]:
df_popPercentage

,Bronx County,Kings County,New York County,Queens County,Richmond County
0,0.170517,0.307526,0.193922,0.271339,0.056696


Brooklyn(kings) and Queens have the most populated area with the least amount of Argentinian restuarants. 

<a id="conclusion"></a>

### Conclusion